<a href="https://colab.research.google.com/github/Djeeone12/Djeeone12.github.io/blob/main/Tarea_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
import io
pd.options.mode.chained_assignment=None
data=pd.read_csv (io.StringIO(uploaded['student_records.csv'].decode('utf-8')))
data

Saving student_records.csv to student_records (1).csv


,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Henry,A,Y,90,85,Yes
1,John,C,N,85,51,Yes
2,David,F,N,10,17,No
3,Holmes,B,Y,75,71,No
4,Marvin,E,N,20,30,No
5,Simon,A,Y,92,79,Yes
6,Robert,B,Y,60,59,No
7,Trent,C,Y,75,33,No


In [ ]:
#define the outcome label
outcome_name=['Recommend']
#define the output set:
outcome_labels=data[outcome_name]
outcome_labels

,Recommend
0,Yes
1,Yes
2,No
3,No
4,No
5,Yes
6,No
7,No


In [ ]:
#define the trainings features lables
featuresNames=['OverallGrade','Obedient','ResearchScore','ProjectScore']
#define the training features set:
trainingFeatures=data[featuresNames]
trainingFeatures

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,90,85
1,C,N,85,51
2,F,N,10,17
3,B,Y,75,71
4,E,N,20,30
5,A,Y,92,79
6,B,Y,60,59
7,C,Y,75,33


In [ ]:
numericFeaturesNames=['ResearchScore','ProjectScore']
categoricalFeaturesNames=['OverallGrade','Obedient']

from sklearn.preprocessing import StandardScaler
ff=StandardScaler ()

#fit on numeric features
ff.fit(trainingFeatures[numericFeaturesNames])

#transforms numeric features
trainingFeatures[numericFeaturesNames]=ff.transform(trainingFeatures[numericFeaturesNames])


#print training features with numeric features  transformed/scaled
trainingFeatures

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,0.899583,1.376650
1,C,N,0.730648,-0.091777
2,F,N,-1.803390,-1.560203
3,B,Y,0.392776,0.772004
4,E,N,-1.465519,-0.998746
5,A,Y,0.967158,1.117516
6,B,Y,-0.114032,0.253735
7,C,Y,0.392776,-0.869179


In [ ]:
trainingFeatures=pd.get_dummies(trainingFeatures,columns=categoricalFeaturesNames)
trainingFeatures

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_B,OverallGrade_C,OverallGrade_E,OverallGrade_F,Obedient_N,Obedient_Y
0,0.899583,1.376650,1,0,0,0,0,0,1
1,0.730648,-0.091777,0,0,1,0,0,1,0
2,-1.803390,-1.560203,0,0,0,0,1,1,0
3,0.392776,0.772004,0,1,0,0,0,0,1
4,-1.465519,-0.998746,0,0,0,1,0,1,0
5,0.967158,1.117516,1,0,0,0,0,0,1
6,-0.114032,0.253735,0,1,0,0,0,0,1
7,0.392776,-0.869179,0,0,1,0,0,0,1


In [ ]:
categoricalFeaturesNames =list(set(trainingFeatures.columns)- set(numericFeaturesNames))
categoricalFeaturesNames

['OverallGrade_E',
 'Obedient_Y',
 'OverallGrade_A',
 'OverallGrade_F',
 'OverallGrade_B',
 'Obedient_N',
 'OverallGrade_C']

In [53]:

# example of training a final regression model
from sklearn.linear_model import LogisticRegression
import numpy as np

logReg = LogisticRegression ()
#fit  the model
model=logReg.fit(trainingFeatures,np.array(outcome_labels['Recommend']))

In [60]:
predictionLabels =model.predict (trainingFeatures)

In [61]:
actualLabels=np.array(outcome_labels['Recommend'])

In [62]:
#Evaluate model perfomance
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print('Accuracy:',float(accuracy_score(actualLabels, predictionLabels))*100,'%')
print('classification Stats:')
print(classification_report(actualLabels,predictionLabels))

Accuracy: 100.0 %
classification Stats:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00         5
         Yes       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [64]:
import joblib
import os

if not os.path.exists('Mpdel'):
  os.mkdir('Model')
if not os.path.exists('Scaler'):
  os.mkdir('Scaler')

joblib.dump(model,r'Model/model.pickle')
joblib.dump(ff,r'Scaler/scaler.pickle')


['Scaler/scaler.pickle']

In [67]:
model =joblib.load(r'Model/model.pickle')
scaler=joblib.load(r'Scaler/scaler.pickle')

In [71]:
new_data=pd.DataFrame([{'Name': 'Nathan','OverallGrade':'F','Obedient':'N','ResearchScore':30,
                        'ProjectScore':20},
                       {'Name': 'Thomas','OverallGrade':'A','Obedient':'Y','ResearchScore':78,
                        'ProjectScore':80}])
new_data =new_data[['Name','OverallGrade','Obedient','ResearchScore','ProjectScore']]
new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore
0,Nathan,F,N,30,20
1,Thomas,A,Y,78,80


In [73]:
#data preparation
prediction_features = new_data[featuresNames]
#scalling
prediction_features[numericFeaturesNames]=scaler.transform(prediction_features[numericFeaturesNames])
prediction_features
#engineering categorical variables
prediction_features =pd.get_dummies(prediction_features,columns=['OverallGrade','Obedient'])
#view feature set
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y
0,-1.127647,-1.430636,0,1,1,0
1,0.494137,1.160705,1,0,0,1


In [74]:
current_categorical_engineered_features =set(prediction_features.columns) - set(numericFeaturesNames)
missing_features = set (categoricalFeaturesNames) - current_categorical_engineered_features
missing_features

{'OverallGrade_B', 'OverallGrade_C', 'OverallGrade_E'}

In [77]:
for feature in missing_features:
  #add zeros sincee features is absent in these adata samples
  prediction_features[feature]= [0]* len(prediction_features)

#views  final feature set
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y,OverallGrade_B,OverallGrade_E,OverallGrade_C
0,-1.127647,-1.430636,0,1,1,0,0,0,0
1,0.494137,1.160705,1,0,0,1,0,0,0


In [79]:
##predict using model 
predictions = model.predict(prediction_features)
#Display  results
new_data['Recommend'] = predictions
new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Nathan,F,N,30,20,No
1,Thomas,A,Y,78,80,Yes
